# **AI Assignment: Connect 4 with MCTS and ID3**

### Assignment Done by:
- David Ventura Mendes de Sá (UP202303580)
- Samuel José Sousa Ventura da Silva (UP202305647)

## 0. Contents
1. Introduction

2. Connect Four  
    **2.1.** Libraries  
    **2.2.** Game Implementation  
    **2.3.** Bitboard vs Matrix  

3. Algorithms    
    **3.1.** Monte Carlo Tree Search (MCTS)   
    **3.2.** Decision Trees (ID3)     
        **3.2.1.** Dataset Generation  

      

4. Algorithms Implementation  
    **4.1.** Libraries   


4. UI Game

6. Results
7. Conclusion


   

## **1. Introduction** ##

In [ ]:
from math import sqrt, log
import random

import game
import mcts
from pygame import gfxdraw
import pygame
from os import environ
import time

## **2. Connect Four** ##

### **2.1. Libraries** ###

In [ ]:
from math import sqrt, log
import random

Escrever alguma coisa sobre as libraries

### **2.2. Game Implementation** ###

Falar da class Bitboard

In [ ]:
class Bitboard:
    def __init__(self):
        self.player1 = 0
        self.player2 = 0
        self.height = [0] * 7
        self.current_player = 1

    # 05 12 19 26 33 40 47
    # 04 11 18 25 32 39 46
    # 03 10 17 24 31 38 45
    # 02 09 16 23 30 37 44
    # 01 08 15 22 29 36 43 
    # 00 07 14 21 28 35 42

    def make_move(self, col):
        
        if col == -1: return

        if self.height[col] >= 6:
            return False

        # Get position
        row = self.height[col]
        bit_position = col * 7 + row

        # Update bitboard
        if self.current_player == 1:
            self.player1 |= (1 << bit_position)
        else:
            self.player2 |= (1 << bit_position)

        # Update heightmap
        self.height[col] += 1

        # Switch to other player1
        self.current_player = 3 - self.current_player
        return True

    def check_player_win(self, player):
        # Diagonal \
        if player == 1:
            board = self.player1
        else:
            board = self.player2

        y = board & (board >> 6)
        if (y & (y >> 2 * 6)):
            return True
        
        # Horizontal
        y = board & (board >> 7)
        if (y & (y >> 2 * 7)):
            return True

        # Diagonal /
        y = board & (board >> 8)
        if (y & (y >> 2 * 8)):
            return True

        # Vertical
        y = board & (board >> 1)
        if (y & (y >> 2)):      
            return True
        return False

    def get_legal_moves(self):
        return [col for col in range(7) if self.height[col] < 6]
    
    def is_over(self):
        return self.check_player_win(1) or self.check_player_win(2) or all(h == 6 for h in self.height)

    def copy(self): # returns deep copy of self
        new_bitboard = Bitboard()
        new_bitboard.player1 = self.player1
        new_bitboard.player2 = self.player2
        new_bitboard.height = self.height.copy()
        new_bitboard.current_player = self.current_player
        return new_bitboard

    def matrix(self):

        matrix = [[0] * 7 for _ in range(6)]

        for bit_position in range(48):
            row = bit_position // 7  
            col = bit_position % 7

            # Check if the bit is set in player1's bitboard
            if self.player1 & (1 << bit_position):
                matrix[col][row] = 1
            # Check if the bit is set in player2's bitboard
            elif self.player2 & (1 << bit_position):
                matrix[col][row] = 2

        return matrix

    def __str__(self):
        # Print the matrix in a readable format
        matrix = self.matrix()
        resul = ""
        for row in matrix:
            for cell in row:
                if cell == 0:
                    resul += "- "
                elif cell == 1:
                    resul += "X "
                elif cell == 2:
                    resul += "O "
            resul += "\n"
        return resul


### **2.3 Bitboard vs Matrix** ###

In [ ]:
##exemplo de codigo que faça o connect4 com matriz ou array nem sei bem

## **3. Algorithms Implementation** ##

### **3.1 Monte Carlo Tree Search (MCTS)** ###

falar um bocado de monte carlo

#### **3.1.1 Class Node** ####

In [ ]:
class Node:
    __slots__ = ['parent', 'move', 'children', 'wins', 'visits']
        
    def __init__(self, parent, move):
        self.parent = parent  # Node
        self.move = move  # move that led to this state
        self.children = {}  # Nodes
        self.wins = 0
        self.visits = 0

    def ucb_score(self, exploration_weight=5):
        if self.visits == 0:
            return float('inf')

        return (self.wins / self.visits) + exploration_weight * sqrt(log(self.parent.visits) / self.visits)

    def expand(self, bitboard):
        children = {Node(self, move) for move in bitboard.get_legal_moves()}
        self.children = children
        return random.choice(list(children))


#### **3.1.2 Class MCTS** ####

In [ ]:
class MCTS:

    def __init__(self, iterations):
        self.iterations = iterations

    def select(self, root, state):
        node = root
        while node.children: 
            node = max(node.children, key=lambda c: c.ucb_score())
            state.make_move(node.move)
        return node, state


    def simulate(self, state):
        moves = state.get_legal_moves()
        while moves:
            move = random.choice(moves)
            state.make_move(move)
            if state.is_over():
                break
            moves = state.get_legal_moves()
        if state.check_player_win(1): return 1
        if state.check_player_win(2): return 2
        return 0
        

    def backpropagate(self, winner, node, state):

        reward = 0 if state.current_player == winner else 1

        while node is not None:
            node.visits += 1
            if winner == 0:
                reward = 0
            else:
                node.wins += reward
                reward = 1 - reward
            node = node.parent


    def search(self, bitboard):
        root = Node(None, None)
        root.expand(bitboard);

        for _ in range(self.iterations):

            state = bitboard.copy()

            leaf, state = self.select(root, state)
            
            # only simulate if its not terminal state
            if not state.is_over():
                leaf = leaf.expand(state)
                state.make_move(leaf.move)
            
            winner = self.simulate(state.copy())
            
            self.backpropagate(winner, leaf, state)

        # stats for the display
        arr = [0] * 14
        for child in root.children:
            arr[child.move] = child.visits
            arr[7+child.move] = child.wins
    
        # return the child with MOST VISITS, we don't use winrate here
        return max(root.children, key=lambda c: c.visits).move, arr


### **3.2 Decision Trees (ID3)** ###

#### **3.2.1. Dataset Generation** ####

Falar um bocado do porque de termos gerado desta maneira

In [ ]:
###codigo do dataset

#### **3.2.2. ID3 Implementation** ####

Qualuer cena

In [ ]:
## codigo do Id3

## **4. Algorithms Implementation** ##

### **4.1. Libraries** ###


In [ ]:
import game
import mcts
from pygame import gfxdraw
import pygame
from os import environ
import time
environ['PYGAME_HIDE_SUPPORT_PROMPT'] = '1'


### **4.2. Nao sei** ###


## **5. User Interface Game** ##


### **5.1. Human vs Human** ###


### **5.2. Human vs MCTS** ###


### **5.3. Human vs ID3** ###


### **5.4. MCTS vs ID3** ###


## **6. Results** ##


## **7. Conclusion** ##
